In [ ]:
import os
import optuna

ID = 13
study = optuna.load_study(study_name=f'experiment_{ID}', storage='sqlite:///{}'.format(os.path.join('..', 'logs', 'experiment', 'studies.db')))

In [ ]:
# If the study is stopped by e.g. system timeout on kaggle or colab the last trial can get stuck in the running state and become "zombie trial".
# Setting fix_running_trials to True will set every running trial to failed one. This may be necessary if study is to be re-run to compute more trials.
# https://github.com/optuna/optuna/issues/685
# https://github.com/optuna/optuna/pull/1214
fix_running_trials = False
running_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.RUNNING])
print(running_trials)
if fix_running_trials:
    for trial in running_trials:
        study._storage.set_trial_state(trial_id=trial._trial_id, state=optuna.trial.TrialState.FAIL)

In [ ]:
len(study.get_trials(deepcopy=False))

In [ ]:
study.best_trial

In [ ]:
optuna.visualization.plot_intermediate_values(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_param_importances(study)